In [ ]:
from collections import namedtuple

from data_preperation.batcher import Batcher
from data_preperation.data import Vocab

from absl import flags
from unified import run_end2end
from unified.model import UnifiedModel
from extractor.model import ExtractorModel
from abstract.model import AbstractModel

flags.FLAGS(["unified"])
from utils import settings

FLAGS = flags.FLAGS

In [ ]:
VOCAB_PATH= '/Users/giladram/Documents/NLP/Final Project/unified-summarization/data/finished_files/vocab'

vocab = Vocab(VOCAB_PATH, FLAGS.vocab_size)  # create a vocabulary

In [ ]:
FLAGS.mode = 'train'
FLAGS.model = 'end2end'
FLAGS.coverage = True

hparam_list = ['model', 'mode', 'eval_method', 'selector_loss_wt', 'inconsistent_loss', 'inconsistent_topk', 'lr',
               'adagrad_init_acc', 'rand_unif_init_mag', 'trunc_norm_init_std', 'max_grad_norm',
               'hidden_dim_selector', 'hidden_dim_rewriter', 'emb_dim', 'batch_size', 'max_art_len', 'max_sent_len',
               'max_dec_steps', 'max_enc_steps', 'coverage', 'cov_loss_wt', 'eval_gt_rouge', 'decode_method']
hps_dict = {}
for key, val in FLAGS.__flags.items():  # for each flag
    if key in hparam_list:  # if it's in the list
        hps_dict[key] = val.value  # add it to the dict
hps = namedtuple("HParams", hps_dict.keys())(**hps_dict)

In [ ]:
batcher = Batcher(FLAGS.data_path, vocab, hps, single_pass=True)

In [ ]:
selector_model = ExtractorModel(hps, vocab)
writer_model = AbstractModel(hps, vocab)
model = UnifiedModel(hps, selector_model, writer_model)
run_end2end.setup_training(model, batcher)

In [ ]:
run_end2end.run_eval(model, batcher)